Pre-requisite:
Create a `.env` file in the root folder based on `.env.sample` and enter the required information.

In [16]:
import os
from dotenv import load_dotenv

load_dotenv(".env")

True

In [17]:
from sqlalchemy import create_engine
import pandas as pd

driver = '{ODBC Driver 17 for SQL Server}'
odbc_str = 'mssql+pyodbc:///?odbc_connect=' \
                'Driver='+driver+ \
                ';Server=tcp:' + os.getenv("SQL_SERVER")+'.database.windows.net;PORT=1433' + \
                ';DATABASE=' + os.getenv("SQL_DB") + \
                ';Uid=' + os.getenv("SQL_USERNAME")+ \
                ';Pwd=' + os.getenv("SQL_PWD") + \
                ';Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'
db_engine = create_engine(odbc_str)

In [18]:
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(model=os.getenv("OPENAI_CHAT_MODEL"),
                      deployment_name=os.getenv("OPENAI_CHAT_MODEL"),
                      temperature=0)

In [31]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain.prompts import FewShotChatMessagePromptTemplate


final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", 
         """
         You are a helpful AI assistant expert in querying SQL Database to find answers to user's question about Categories, Products and Orders.
         'Product' table contains information about products including product name, description, product price and product category.
         'Category' table contains information about categories including category name and description. Each Product is mapped to a Category.
         'Orders' table contains information about orders placed by customers including 
         quantity or number of products ordered,
         expected delivery date and actual delivery date of the Order in the location
         and the status of the order. 
         Order status = 'Processing' means the order is being processed by seller and not yet shipped,
         Order status = 'Shipped' means the order is shipped by the seller and is on the way to the customer,
         Order status = 'Completed' means the order is delivered to the customer, and 
         Order status = 'Cancelled' means the order is cancelled by the customer.

         Identify the table and column names in the database that contain the information needed to answer the user's question. 
         Make a summation of the column name including 'quantity' if the question is about number of product.
        """
         ),
        ("human: ", "{question}\n ai: "),
    ]
)

In [20]:
from langchain.agents import AgentType, create_sql_agent
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

db = SQLDatabase(db_engine)

sql_toolkit = SQLDatabaseToolkit(db=db, llm=llm)
sql_toolkit.get_tools()

sqldb_agent = create_sql_agent(
    llm=llm,
    toolkit=sql_toolkit,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    top_p=0
)

In [32]:
sqldb_agent.run(final_prompt.format(question ="How many Kitchen products were sold in current month?"))




> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Category, Orders, Product, sysdiagrams
Thought:The relevant tables in the database are Category, Orders, and Product. I should query the schema of these tables to find the relevant columns.
Action: sql_db_schema
Action Input: "Category, Orders, Product"
Observation: 
CREATE TABLE [Category] (
	[CategoryId] INTEGER NOT NULL IDENTITY(1,1), 
	[CategoryName] VARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	CONSTRAINT [PK__Category__19093A0B02B89A54] PRIMARY KEY ([CategoryId])
)

/*
3 rows from Category table:
CategoryId	CategoryName
1	Kitchen
2	Sports
3	Gardening
*/


CREATE TABLE [Orders] (
	[OrderId] INTEGER NOT NULL IDENTITY(1,1), 
	[ProductId] INTEGER NULL, 
	[OrderCreationDate] DATETIME NOT NULL, 
	[ProductOrderedQuantity] INTEGER NOT NULL, 
	[OrderStatus] VARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[OrderDeliveryLocation] VARCHAR(max) COLLATE SQL_Latin1_Gene

'2'

In [22]:
sqldb_agent.run(final_prompt.format(question="How many orders have not been shipped yet?"))



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Category, Orders, Product, sysdiagrams
Thought:I see that there are four tables in the database: Category, Orders, Product, and sysdiagrams. I should query the Orders table to find the number of orders that have not been shipped yet.
Action: sql_db_schema
Action Input: "Orders"
Observation: 
CREATE TABLE [Orders] (
	[OrderId] INTEGER NOT NULL IDENTITY(1,1), 
	[ProductId] INTEGER NULL, 
	[OrderCreationDate] DATETIME NOT NULL, 
	[ProductOrderedQuantity] INTEGER NOT NULL, 
	[OrderStatus] VARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[OrderDeliveryLocation] VARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[OrderDeliveryExpectedDate] DATETIME NOT NULL, 
	[OrderDeliveryActualDate] DATETIME NULL, 
	CONSTRAINT [PK__Orders__C3905BCF61D2B5A4] PRIMARY KEY ([OrderId]), 
	CONSTRAINT [FK__Orders__ProductI__3587F3E0] FOREIGN KEY([ProductId]) REFERENCES [Product] ([ProductId])


'There are 4 orders that have not been shipped yet.'

In [23]:
sqldb_agent.run(final_prompt.format(question="How many orders were delivered late last month?"))



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Category, Orders, Product, sysdiagrams
Thought:I see that there are four tables in the database: Category, Orders, Product, and sysdiagrams. I should query the schema of the Orders table to see what columns are available.
Action: sql_db_schema
Action Input: "Orders"
Observation: 
CREATE TABLE [Orders] (
	[OrderId] INTEGER NOT NULL IDENTITY(1,1), 
	[ProductId] INTEGER NULL, 
	[OrderCreationDate] DATETIME NOT NULL, 
	[ProductOrderedQuantity] INTEGER NOT NULL, 
	[OrderStatus] VARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[OrderDeliveryLocation] VARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[OrderDeliveryExpectedDate] DATETIME NOT NULL, 
	[OrderDeliveryActualDate] DATETIME NULL, 
	CONSTRAINT [PK__Orders__C3905BCF61D2B5A4] PRIMARY KEY ([OrderId]), 
	CONSTRAINT [FK__Orders__ProductI__3587F3E0] FOREIGN KEY([ProductId]) REFERENCES [Product] ([ProductId])
)

/*
3 rows

'There was 1 order delivered late last month.'

In [24]:
sqldb_agent.run(final_prompt.format(question="What are the top 3 products sold last month? Provide the product name and the quantity sold."))



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Category, Orders, Product, sysdiagrams
Thought:I can query the Product and Orders tables to find the top 3 products sold last month. I should check the schema of these tables to see the relevant columns.
Action: sql_db_schema
Action Input: "Product, Orders"
Observation: 
CREATE TABLE [Orders] (
	[OrderId] INTEGER NOT NULL IDENTITY(1,1), 
	[ProductId] INTEGER NULL, 
	[OrderCreationDate] DATETIME NOT NULL, 
	[ProductOrderedQuantity] INTEGER NOT NULL, 
	[OrderStatus] VARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[OrderDeliveryLocation] VARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[OrderDeliveryExpectedDate] DATETIME NOT NULL, 
	[OrderDeliveryActualDate] DATETIME NULL, 
	CONSTRAINT [PK__Orders__C3905BCF61D2B5A4] PRIMARY KEY ([OrderId]), 
	CONSTRAINT [FK__Orders__ProductI__3587F3E0] FOREIGN KEY([ProductId]) REFERENCES [Product] ([ProductId])
)

/*
3 rows from Ord

"The top 3 products sold last month are 'Seed' with a quantity of 20, 'Seed' with a quantity of 5, and 'Ladle' with a quantity of 5."